In [1]:
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()

'C:\\spark\\SPARK_HOME'

In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
conf = pyspark.SparkConf().setAppName('appName').setMaster('local') #Основным механизмом настройки в Spark 
                                                                    #является класс SparkConf.
#conf = new SparkConf()
#conf.set("spark.app.name", "Му Spark Арр")
#conf.set("spark.master", "local[4]")
#conf.set("spark.ui.port", "36000") # Переопределить порт по умолчанию

sc = pyspark.SparkContext(conf=conf) # Создать SparkContext с данной конфигурацией
spark = SparkSession(sc)

In [4]:
sc

<SparkContext master=local appName=appName>

# Программирование операций с RDD

In [3]:
lines = sc.textFile("C:\spark\SPARK_HOME\README.md") #Создание набора RDD путем считывания файла

In [4]:
print lines.count() #вызов действия (не создает новый RDD)
print lines.first()

104
# Apache Spark


In [5]:
lines.persist()

C:\spark\SPARK_HOME\README.md MapPartitionsRDD[1] at textFile at <unknown>:0

In [11]:
pythonLines = lines.filter ( lambda line: "Python" in line) # вызов преобразования (создает новое RDD)

In [12]:
pythonLines.persist # сохранение RDD pythonLines в памяти

<bound method PipelinedRDD.persist of PythonRDD[8] at RDD at PythonRDD.scala:53>

# Создание RDD

In [13]:
lines = sc.parallelize(["pandas", "i like pandas"]) #создание RDD способ 1

In [15]:
lines = sc.textFile("C:\spark\SPARK_HOME\README.md") #Создание набора RDD путем считывания файла (cпособ 2)

# Преобразования

In [ ]:
*RDD_out = in_RDD.filter (lambda x: "word" in x) #фильтрация

*RDD_out = in_RDD.map(lambda x: x*x) # функция map принимает в качестве аргумента другую функцию и применяет ее к каждому 
                                      # элементу RDD 
                                       
*RDD_out = in_RDD.flatMap(lambda x: x.split(" ")) # функция flatMap принимает в качестве аргумента другую функцию и применяет 
                                                  #  ее к каждому элементу RDD. Но вместо единственного элемента наша функция 
                                                  #  должна вернуть итератор для обхода возвращаемых значений новых элементов.
                                                  #  Однако вместо набора RDD итераторов flatMap() возвращает набор RDD с 
                                                 #  элементами, получаемыми с применением всех итераторов.                      
                                                    
*RDD_final = RDD_1.union(RDD_2) #объединение двух RDD

*RDD.distinct() #возвращает RDD без повторов элементов (только уникальные элементы)

*RDD1.intersection(RDD2) # пересечение элементов двух RDD

*RDD1.substract(RDD2) # вычитание из множества элементов RDD1 множества элементов RDD2

*RDD1.cartesian(RDD2) #декартово произведение двух RDD

# Действия

In [ ]:
*RDD.count() #число элементов

*for line in RDD.take(n): #извлечение n первых элементов RDD
    print line
    
*RDD.collect() # извлечение всех элеметов rdd в память

*RDD.reduce(lambda x,y: x+y) #принимает функцию, оперирующую двумя элементами данного RDD в качестве аргумента и возвращает 
                            #элемент того же типа
                              
*RDD.fold(0)(lambda x,y: x+y) #то же самое что и reduce, но с указанием нулевого значения

*RDD.aggregate((0,0), #начальное нулевое значение типа, который следует вернуть 
               (lambda acc,value: (acc[0]+value, acc[1]+1)), # функция для объединения двух элементов с накопителем
               (lambda acc1, acc2: (acc1[0]+acc2[0], acc1[1]+acc2[1])) #функция для объединения результатов двух узлов
              )
              
*RDD.top() #возвращает верхние элементы RDD

*RDD.takeSample(withReplacement, num, seed) #возвращает случайные элементы коллекции, num - число элементов, 
                                            # withReplacement(true or false) - c заменой или без
                                             
*RDD.foreach(func) # выполнить действие над всеми элементами в наборе RDD, но без возврата результата в программу-драйвер

*RDD.countByValue () # возвращает отображение каждого уникального значения на его число (определяющее, сколько раз это значение 
                      # встречается в наборе)

# Передача функций в SPARK

In [ ]:
word = RDD.filter(lambda s: "error" in s) # c помощью lambda

In [ ]:
def containsError(s): # c помощью определения пользовательской функции
    return "error" in s
word = rdd.filter(containsError)

# Работа с парами ключ/значение

In [ ]:
RDD.mapValues(func) # применить func к каждому значению без изменения ключа
RDD.keys() # получить RDD из ключей
RDD.values() # получить RDD из значений
RDD.flatMapValues(func) # Применить функцию, возвращающую итератор для каждого значения в наборе пар и создающую 
                        # для каждого возвращаемого элемента пару ключ/значение со старым ключом. 
#АГРЕГИРОВАНИЕ
RDD.combineByKey(createCombiner, mergeValue, mergeCombiners,partitioner) #Объединить значения с одинаковыми ключами и получить результат другого типа
#createCombiner(value) - пользовательская функция, запускающаяся при встрече с новым ключом (создает начальное значение для счетчика)
#mergeValue(acc,value) - пользовательская функция (с текущим значением счетчика и новым значением), запускающаяся при встрече с ключом, 
            #который уже встречался
#mergeCombiners(acc1,acc2) - пользовательская функци, запускающаяся при объединении результатов обработки разных разделов если обнаружится
            #несколько значений аккумулятора для одного и того же ключа
#partitioner - объект управления распределением 
RDD.reduceByKey(func) # объединить значения с одинаковыми ключами (применить func к значениям с одинаковыми ключами)

#Настройка уровня параллелизма
sc.parallelize(data).reduceByKey(lambda x, y: x + y, 10) #второй параметр функции определяет уровень параллелизма
RDD.repartion() #перераспределение данных
RDD.coalesce() #оптимизированная версия repartion
RDD.partitions.size() #размер RDD
RDD.partitionBy(n, func) #разбиение RDD на n разделов, определяет, сколько параллельных заданий будет создаваться при выполнении
                    #операций над этим набором RDD, func - функция хэширования

#Группировка данных
RDD.groupByKey() # сгруппировать значения с одинаковыми ключами (на выходе каждому ключу соответствует кортеж значений 
                     # имеющих этот ключ)
RDD.groupBy(func) #применяет func к каждому элементу RDD, полученный результат использует как ключ

RDD1.cogroup(RDD2) #операция выполняется над двумя наборами RDD с ключами одного типа К и значениями типа V и W, она 
                    #возвращает набор типа RDD [ (К, ( Iterable [V], Iterable [W]))]

#Соединения
RDD1.join(RDD2) #INNER JOIN
RDD1.leftOuterJoin(RDD2) #внешнее левостороннее соединение
RDD1.rightOuterJoin(RDD2) #внешнее правостороннее соединение

#Сортировка
RDD.sortByKey(ascending=True, numPartitions=None, keyfunc = lambda x: str(x)) # отсортировать по значениям ключей
                                                                              #ascending - по возрастанию или убыванию

In [33]:
#Пример. Подсчет слов
rdd = sc.textFile("C:\spark\SPARK_HOME\README.md")
words = rdd.flatMap(lambda x: x.split(" ")) #создание RDD, где каждый элемент отдельное слово
result = words.map(lambda x: (x, 1)).reduceByKey(lambda x, y: x + y) 
print result.collect()[:5]

[(u'', 72), (u'project.', 1), (u'help', 1), (u'when', 1), (u'Hadoop', 3)]


In [4]:
# Пример 4. 12. Вычисление среднего значения для каждого ключа с помощью combineByKey() в Python
nums = sc.parallelize([(0,1), (0,2), (1,3), (1,4)])
sumCount = nums.combineByKey(
(lambda x: (x,1)), #x - соответсв значение в паре ключ-значение
(lambda x, y: (x[0] + y, x[1] + 1)),
(lambda x, y: (x[0] + y[0], x[1] + y[1])))
print(sumCount.collect())
print(sumCount.map(lambda (key,(summa, count)): (key, summa/float(count))).collect())

[(0, (3, 2)), (1, (7, 2))]
[(0, 1.5), (1, 3.5)]


# Действия над наборами RDD c парами ключ-значение

In [ ]:
rdd.countByKey() #Подсчет числа элементов  для каждого ключа
rdd.collectAsMap() # Извлечение данных в виде  словаря
rdd.lookup(key) # Извлечение всех значений, связанных с указанным ключом

# ЗАГРУЗКА И СОХРАНЕНИЕ ДАННЫХ

# Текстовые данные

In [ ]:
input = sc.textFile("C:\spark\SPARK_HOME\README.md") # загрузка текстового файла, каждое слово - отдельный элемент RDD

In [10]:
input = sc.wholeTextFiles("C:\spark\SPARK_HOME\\") #загрузка всех текстовых файлов, содержащихся в директории SPARK_HOME, 
                                                    #в формате ключ-значение, где ключ - имена файлов
input.keys().collect()

[u'file:/C:/spark/SPARK_HOME/1.txt',
 u'file:/C:/spark/SPARK_HOME/LICENSE',
 u'file:/C:/spark/SPARK_HOME/NOTICE',
 u'file:/C:/spark/SPARK_HOME/README.md',
 u'file:/C:/spark/SPARK_HOME/RELEASE']

In [ ]:
result.saveAsTextFile(outputFile) #сохранение RDD result в текстовый файл outputFile

# JSON

In [ ]:
import json
input = sc.textFile("C:\spark\SPARK_HOME\name_file.json") #загрузка json файла как текстового (одна строка - одна JSON запись)
data = input.map(lambda x: json.loads(x)) #парсер json файла

In [ ]:
data.map(lambda x: json.dumps(x)).saveAsTextFile(outputFile) #сохранение данных в json формате

# CSV

In [ ]:
import csv
import StringIO
#загрузка одного файла
def loadRecord(line):
    """Парсинг строки CSV"""
    input = StringIO.StringIO(line)
    reader = csv.DictReader(input, fieldnames=["name", "favouriteAnimal"])
    return reader.next()
input = sc.textFile("C:\spark\SPARK_HOME\name_file.csv").map(loadRecord)

In [ ]:
#загрузка всех файлов целиком
def loadRecords(fileNameContents):
    """Загружает все записи из заданного файла"""
    input = StringIO.StringIO(fileNameContents[1])
    reader = csv.DictReader(input, fieldnames=["name", "favoriteAnimal"])
    return reader
fullFileData = sc.wholeTextFiles("C:\spark\SPARK_HOME\\").flatMap(loadRecords)

In [ ]:
#запись в файл
def writeRecords(records): 
    """Записывает в файл строки CSV"""
    output = StringIO.StringIO()
    writer = csv.DictWriter(output, fieldnames= ["name", "favoriteAnimal"])
    for record in records:
        writer.writerow(record)
    return [output.getvalue()]

# SequenceFiles

SequenceFiles - популярный формат файлов, используемый в Hadoop, состоящих из пар ключ/значение.

sequenceFile(path, keyClass, valueClass, minPartitions) - функция загрузки

path - путь к файлу

keyClass - формат ключей

valueClass - формат значений


In [ ]:
data = sc.sequenceFile(inFile, "org.apache.hadoop.io.Text", "org.apache.hadoop.io.IntWritаblе")

In [ ]:
data = sc.parallelize(list(("Panda", 3), ("Кау", 6), ("Snail", 2)))
data.saveAsSequenceFile(outputFile)  #сохранение в формате sequenceFiles

# Объектные файлы

In [ ]:
input = sc.pickleFile("C:\spark\SPARK_HOME\file_name")
input.saveAsPickleFile(outputfile)

# РАЗДЕЛЯЕМЫЕ ПЕРЕМЕННЫЕ

# 1) Аккумуляторы

In [11]:
file = sc.textFile('C:\spark\SPARK_HOME\README.md')

blankLines = sc.accumulator(0) # Создать Accumulator[Int], инициализированный нулем

def extractCallSigns(line):
    global blankLines # Получить доступ к глобальной переменной
    if (line == ""):
        blankLines += 1
    return line.split(" ")

callSigns = file.flatMap(extractCallSigns)
callSigns.saveAsTextFile("C:\spark\SPARK_HOME\\" + "callsigns")
print "Blank lines: %d" % blankLines.value #доступ к значению аккумулятора

Blank lines: 40


# 2) Широковещательные переменные

Широковещательные переменные (bгoadcast vmiaЬles) - позволяют программам эффективно передавать большие значения, доступные только для чтения, всем рабочим узлам для использования в операциях Spark.
Переменная будет отправлена каждому узлу лишь один раз и должна использоваться только для чтения (изменения в переменной не будут передаваться другим узлам).

In [ ]:
# Пример 6.7: Поиск страны с помощью широковещательной переменной в Python
# Поиск страны по позывному в наборе RDD coпtactCouпts. С этой целью загружается массив кодов стран с соответствующими
# префиксами позывных.
signPrefixes = sc.broadcast(loadCallSignTable()) #cоздание широковещательной переменной

def processSignCount (sign count, signPrefixes) :
    country = lookupCountry(sign_count[0], signPrefixes.value) #метод value предоставляет доступ к значению широковещательной переменной
    count = sign_count[1]
    return (country, count)
countryContactCounts = (contactCounts.map(processSignCount).reduceByKey((lambda x, y: x+y)))
countryContactCounts.saveAsTextFile(outputDir + "/countries.txt")

# Взаимодейсвтие с внешними программами:

Наборы RDD в Spark имеют метод pipe (), благодаря которому можно писать задания для Spark на любом языке, при условии что
они будут записывать данные в стандартные потоки ввода/вывода Unix и читать их оттуда.

# Числовые операuии над наборами RDD

Все описательные статистики вычисляются за один проход по данным и возвращаются методом stats () в виде объекта StatsCounter.

RDD.count() - Число элементов в наборе

RDD.mean() - Среднее значение по элементам

RDD.sum() - Общую сумму элементов

RDD.max() - Максимальное значение

RDD.min() - Минимальное значение

RDD.variance() - Дисперсию элементов

RDD.sampleVariance() - Дисперсию для выборки элементов

RDD.stdev() - Стандартное отклонение

RDD.sampleStdev() - Стандартное отклонение для выборки элементов

stats = RDD.stats() - Вычисляет все статистки сразу же

mean = stats.mean() - доступ к статистикам

# Выполнение в кластере

Драйвер экспортирует информацию о выполняемом приложении Spark через веб-интерфейс: http://localhost:4040 

Запуск приложения в кластере с помощью Spark:

1. Пользователь вызывает сценарий spark-submit, чтобы запустить приложение.

2. spark-submit запускает· программу-драйвер и вызывает метод main (), указанный пользователем.

3. Программа-драйвер связывается с диспетчером кластера и запрашивает у него ресурсы для запуска исполнителей.

4. Диспетчер кластера запускает исполнителей от имени программы- драйвера.

5. Процесс драйвера выполняет инструкции в пользовательском приложении. Опираясь на действия и преобразования наборов RDD в программе, драйвер посылает задания исполнителям.

6. Исполнители выполняют задания, получают и сохраняют результаты.

7. Если драйвер выполняет выход из метода main () или вызывает SparkContext. stop (), Spark останавливает исполнителей и освобождает ресурсы, возвращая их диспетчеру кластера.

In [30]:
%run -i C:\spark\SPARK_HOME\examples\src\main\python\wordcount.py C:\spark\SPARK_HOME\README.md #запуск скрипта python (.py)

: 72
project.: 1
help: 1
when: 1
Hadoop: 3
MLlib: 1
"local": 1
./dev/run-tests: 1
including: 4
computation: 1
file: 1
high-level: 1
find: 1
web: 1
Shell: 2
cluster: 2
Kubernetes: 1
also: 5
using:: 1
Big: 1
guidance: 2
run:: 1
Scala,: 1
Running: 1
should: 2
environment: 1
to: 17
only: 1
module,: 1
given.: 1
rich: 1
directory.: 1
Apache: 1
Interactive: 2
sc.parallelize(range(1000)).count(): 1
Building: 1
do: 2
guide,: 1
return: 2
which: 2
Programs: 1
Many: 1
Enabling: 1
built,: 1
YARN,: 1
R,: 1
using: 3
Example: 1
scala>: 1
Once: 1
-DskipTests: 1
Spark"](http://spark.apache.org/docs/latest/building-spark.html).: 1
and: 10
Because: 1
cluster.: 1
name: 1
Testing: 1
Streaming: 1
./bin/pyspark: 1
SQL: 2
through: 1
GraphX: 1
them,: 1
guide](http://spark.apache.org/contributing.html): 1
[run: 1
resource-managers/kubernetes/integration-tests/README.md: 1
development: 1
abbreviated: 1
set: 2
For: 3
Scala: 2
##: 9
the: 23
thread,: 1
library: 1
see: 3
individual: 1
examples: 2
MASTER: 1
runs.: 1
[

# SPARK-SUBMIT

In [ ]:
#запуск с командной строки ANACONDA через сценарий spark-submit (локально)
spark-submit C:\spark\SPARK_HOME\examples\src\main\python\wordcount.py C:\spark\SPARK_HOME\README.md
#запуск с командной строки ANACONDA как скрипт python
python C:\spark\SPARK_HOME\examples\src\main\python\wordcount.py C:\spark\SPARK_HOME\README.md

## Пример 7 .З •:• Общий синтаксис командной строки spark-submit

bin/spark-submit [options] <арр jar I python file> [арр options]

1) [options] - список флагов для spark-subrnit; получить полный список флагов можно, выполнив команду spark-subrnit --help;

2) <арр jar I python file> - имяJАR-файла или сценария на Python, содержащего точку входа в приложение;

3) [арр options] - параметры для передачи приложению; если метод main () программы предусматривает анализ параметров командной строки, он увидит только параметры [ арр options], флаги, предназначенные для spark-submit, будут ему недоступны.

## Таблица 7. 1. Возможные значения для флага --master сценария spark-submit

spark:/host:port  - Адрес кластера Spark Standalone с указанным портом. По умолчанию диспетчеры кластеров 
                    Spark Standalone используют порт 7077
            
mesos://host:port - Адрес кластера Mesos с указанным портом. По умолчанию диспетчеры кластеров Mesos используют порт 5050
        
yarn  -  Адрес кластера YARN. Перед запуском YARN необходимо определить переменную окружения HADOOP CONF DIR, 
         в которой указать путь к каталогу с настройками Hadoop, описывающими параметры кластера
    
local -  Запуск в локальном режиме на одном ядре

local[N] -  Запуск в локальном режиме на N ядрах

local[*] -  Запуск в локальном режиме на всех ядрах, имеющихся на машине

# Диспетчер кластера Spark Standalone

Запустить кластер вручную можно с помощью сценария spark-class в подкаталоге bin/ каталога установки Spark. 

На ведущей машине выполните команду:
bin/spark-class org.apache.spark.deploy.master.Master

Затем на ведомых машинах:
bin/spark-class org.apache.spark.deploy.worker.Worker spark://masternode:7077
(где masternode - сетевое имя ведущей машины)

Точно таким же способом можно запустить на кластере spark-shell или pyspark, передав флаг --master:

spark-shell --rnaster spark://masternode:7077

pyspark --rnaster spark://masternode:7077

# SPARK SQL

Spark SQL определяет специальный тип RDD с именем SchernaRDD. Класс SchernaRDD представляет набор RDD объектов Row, каждый из которых представляет отдельную запись.

Чтобы задействовать Spark SQL, необходимо создать объект HiveContext (или SQLContext, когда поддержка Hive недоступна), опираясь на объект SparkContext. Этот объект контекста предоставляет дополнительные функции для получения и обработки данных. С помощью HiveContext можно создавать наборы данных SchemaRDD (DataFrame), представляющие структурированные данные, и оперировать ими с применением запросов SQL или обычных операций над RDD, таких как map ()

In [9]:
# Импортировать Spark SQL
from pyspark.sql import HiveContext, Row
#from pyspark.sql import SQLContext, Row # Или если не должно быть зависимостей от Hive
#sqlCtx = SQLContext(sc) #Создание экземпляра контекста SQL в Python
hiveCtx = HiveContext(sc)

In [28]:
# Пример 9.11. Простой запрос SQL
inputFile = "testweet.json"
#inputFile = "C:\spark\SPARK_HOME\README.md"
input = hiveCtx.read.json(inputFile)
input.registerTempTable("tweets") # Зарегистрировать схему RDD
topTweets = hiveCtx.sql("""SELECT text, retweetCount FROM tweets ORDER BY retweetCount LIMIT 10""") # Выбрать сообщения по retweetCouпt
topTweets.collect()

[Row(text=u'Adventures With Coffee, Code, and Writing.', retweetCount=0)]

In [31]:
topTweets.show() #выводтаблицы в формате sql

+--------------------+------------+
|                text|retweetCount|
+--------------------+------------+
|Adventures With C...|           0|
+--------------------+------------+



Наборы SchemaRDD являются самыми обычными наборами данных, поэтому к ним могут применяться любые преобразования, такие как
map () и filter (). Но, кроме этого, они обладают дополнительными функциональными возможностями. Самая важная из них - возможность зарегистрировать любой набор SchemaRDD как временную таблицу и выполнять запросы к ней с вызовом метода HiveContext.sql () или SQLContext.sql ().

# Операции над объектами Row

Объекты Row представляют записи внутри наборов SchemaRDD и фактически являются массивами полей фиксированной длины.
В Python отсутствует явная типизация. Программа может просто обращаться к i-му элементу как row [i]. Кроме того, объекты Row в Python поддерживают доступ к полям по именам, в форме row.column_name

In [35]:
topTweetText = topTweets.rdd.map(lambda row: row.text) #обращение к текстовому полю путем перевода dataframe в rdd
topTweetText.collect() 

[u'Adventures With Coffee, Code, and Writing.']

In [40]:
df_topTweet = topTweetText.map(lambda x: (x, )).toDF() #создание df из rdd, способ 1
df_topTweet.show()

+--------------------+
|                  _1|
+--------------------+
|Adventures With C...|
+--------------------+



In [41]:
happyPeopleRDD = sc.parallelize([Row(name="holden", favouriteBeverage="coffee")]) #создание df из rdd способ 2
happyPeopleSchemaRDD = hiveCtx.createDataFrame(happyPeopleRDD)
happyPeopleSchemaRDD.show()

+-----------------+------+
|favouriteBeverage|  name|
+-----------------+------+
|           coffee|holden|
+-----------------+------+



# Функции, определяемые польэователем в SPARK_SQL

In [52]:
hiveCtx.registerFunction("strLenPython", lambda x: len (x), pyspark.sql.types.IntegerType()) #определение пользовательской функии
lengthSchemaRDD = hiveCtx.sql("SELECT strLenPython('text') FROM tweets LIMIT 10")
lengthSchemaRDD.show()

+------------------+
|strLenPython(text)|
+------------------+
|                 4|
+------------------+



# Машинное обучение с MLlib

# пример. Спам/не спам

In [1]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.feature import HashingTF #частота слов
from pyspark.mllib.classification import LogisticRegressionWithSGD 

In [4]:
spam = sc.textFile("spam.txt")
normal = sc.textFile("ham.txt")
# Создать экземпляр HashingTF для отображения текста электронных писем в векторы с 10 ООО признаков.
tf = HashingTF(numFeatures = 10000)
# Разбить каждое электронное пистьмо на слова и каждое слово отобразить в один признак.
spamFeatures = spam.map(lambda email: tf.transform(email.split(" ")))
normalFeatures = normal.map(lambda email: tf.transform(email.split(" ")))

In [5]:
# Создать наборы данных LabeledPoint для примеров, дающих положительную реакцию (спам) и отрицательную (обычные письма).
positiveExamples = spamFeatures.map(lambda features: LabeledPoint(1, features))
negativeExamples = normalFeatures.map(lambda features: LabeledPoint(0, features))
trainingData = positiveExamples.union(negativeExamples)
trainingData.cache() # Кэшировать, потому что алгоритм Logistic Regression является циклическим.

UnionRDD[6] at union at <unknown>:0

In [6]:
# Выполнить логистическую регрессию методом SGD.
model = LogisticRegressionWithSGD.train(trainingData)
#Проверить положительный экземпляр (спам) и отрицательный (неспам).
#Сначала применить то же преобразование HashingTF, чтобы получить векторы признаков, затем применить модель.
posTest = tf.transform( "О М G GET cheap stuff Ьу sending money to ... ".split (" "))
negTest = tf.transform( "Hi Dad, I started studying Spark the other ... ".split(" "))
print "Prediction for positive test example: %g" %model.predict(posTest)
print "Prediction for negative test example: %g" %model.predict(negTest)

Prediction for positive test example: 1
Prediction for negative test example: 0


# Типы данных в MLlib

1) Vector - вектор в математическом смысле. MLlib поддерживает обе разновидности векторов: плотные, хранящие все элементы, и разреженные, хранящие только ненулевые значения для экономии памяти. Векторы могут конструироваться с помощью класса mllib.linalg.Vector

2) LabeledPoint - маркированная точка в пространстве данных для использования в алгоритмах обучения, таких как классификация и регрессия. Включает вектор признаков и маркер (являющийся вещественным числом). Определение находится в пакете mllib.regression package

3) Rating - оценка продукта пользователем, используемая в пакете mllib.recommendation для определения рекомендаций

4) семейство классов Model - все модели типа Model являются результатом работы алгоритма обучения и обычно имеют метод predict() для применения модели к новой точке или к набору RDD новых точек данных

# Векторы

In [8]:
from numpy import array
from pyspark.mllib.linalg import Vectors

# Создать плотный вектор <1.0, 2.0, 3.0>
denseVecl = array([1.0, 2.0, 3.0]) # в MLlib можно передавать непосредственно массивы NumPy

denseVec2 = Vectors.dense([1.0, 2.0, 3.0]) # .. или использовать класс Vectors

# Создать разреженный вектор <1.0, О.О, 2.0, О.О>; соответствующие методы принимают только размер вектора (4) и
# индексы с ненулевыми элементами. Исходные данные можно передать в виде словаря или как два риска - индексов и значений.
sparseVecl = Vectors. sparse ( 4, {0: 1.0, 2: 2.0})
sparseVec2 = Vectors.sparse(4, [0, 2], [1.0, 2.0])

# Извлечение признаков (mllib. feature)

# TF-IDF

Term Frequency - lnverse Document Frequency (частота слова-обратная частота документа), или TF-IDF - это простой алгоритм
создания векторов признаков для текстовых документов (таких как веб-страницы). Он вычисляет для каждого слова в каждом документе две статистики: частоту слова (Term Frequency, TF), то есть сколько раз встретилось это слово в документе, и обратную частоту документа (Inverse Document Frequency, IDF), оценивающую, как (не)часто встречается слово во всей коллекции документов. Произведение этих значений, TF х IDF, показывает, насколько важным является слово для конкретного документа (то есть слово может часто использоваться в данном документе, но редко во всей коллекции в целом)

В библиотеке MLlib имеются два алгоритма для вычисления оценки TF-IDF: HashingTF и IDF

In [11]:
from pyspark.mllib.feature import HashingTF
sentence = "hello hello world"
words = sentence.split(" ") #Разбить sentence на список слов
tf = HashingTF(10000) # Создать вектор размера S = 10000
tf.transform(words) #вычисление TF

SparseVector(10000, {3741: 2.0, 9145: 1.0})

In [22]:
from pyspark.mllib.feature import HashingTF, IDF
# Прочитать множество текстовых файлов в векторы TF
data = "C:\spark\SPARK_HOME"
rdd = sc.wholeTextFiles(data).map(lambda (name, text): text.split())
tfVectors = tf.transform(rdd).cache() # Преобразовать сразу весь набор документов RDD

# Вычислить IDF, затем векторы TF-IDF
idf = IDF ()
idfModel = idf .fit (tfVectors)
tfidfVectors = idfModel.transform(tfVectors) #TF-IDF вектор

# Масштабирование

Класс StandardScaler из MLlib, выполняет масштабирование по средним значениям и стандартным отклонениям

In [26]:
from pyspark.mllib.feature import StandardScaler
vectors = [Vectors.dense([-2.0, 5.0, 1.0]), Vectors.dense([2.0, 0.0, 1.0])]
dataset = sc.parallelize(vectors)
scaler = StandardScaler(withMean=True, withStd=True)
model = scaler.fit(dataset)
result = model.transform(dataset)
print result.collect()

[DenseVector([-0.7071, 0.7071, 0.0]), DenseVector([0.7071, -0.7071, 0.0])]


# Нормализация векторов

In [28]:
from pyspark.mllib.feature import Normalizer
vectors = [Vectors.dense([-2.0, 5.0, 1.0]), Vectors.dense([2.0, 0.0, 1.0])]
dataset = sc.parallelize(vectors)
result = Normalizer().transform(dataset) #приводит результат к единичной норме
print result.collect()

[DenseVector([-0.3651, 0.9129, 0.1826]), DenseVector([0.8944, 0.0, 0.4472])]


# Word2Vec

Word2Vec 1 (https://code.google.com/p/word2vec/) - это алгоритм выделения признаков для текста на основе нейронных сетей, который можно использовать для подготовки данных к обрабьтке следующими за ним алгоритмами. Фреймворк Spark включает реализацию этого алгоритма в виде класса mllib. feature. Word2Vec.

Для обучения Word2Vec следует передать коллекцию документов, представленную объектами-коллекциями Iterable строк (по одному слову в строке). Так же как для алгоритма TF-IDF, рекомендуется нормализовать слова (например, привести все символы к нижнему
регистру и удалить из текста знаки пунктуации и числа). Результатом обучения (вызовом Word2Vec .fit (rdd)) является объект Word2VecModel, метод transform() которого можно использовать для преобразования каждого слова в вектор. Обратите внимание, что размер модели в алгоритме Word2Vec равен числу слов в словаре, умноженному на размер вектора (по умолчанию 100).

In [32]:
from pyspark.mllib.feature import Word2Vec

inp = sc.textFile("C:\spark\SPARK_HOME\data\mllib\sample_lda_data.txt").map(lambda row: row.split(" "))

word2vec = Word2Vec()
model = word2vec.fit(inp)

synonyms = model.findSynonyms('1', 5)

for word, cosine_distance in synonyms:
    print("{}: {}".format(word, cosine_distance))

2: 0.313273698092
4: 0.207959577441
0: 0.202663198113
3: 0.107770711184
9: -0.0441659018397


# Статистики

Библиотека MLlib предлагает возможность вычисления широко используемых статистических характеристик для наборов RDD с помощью методов класса mllib.stat.Statistics, таких как: 

1) Statistics.colStats(rdd) - вычисляет статистические характеристики для векторов в наборе RDD: минимальное, максимальное, среднее и дисперсию для каждого столбца во множестве векторов. Может использоваться для получения широкого разнообразия статистик за один проход;

2) Statistics.corr(rdd, method) - вычисляет матрицу корреляций между столбцами в наборе векторов, с использованием алгоритма Пирсона (Pearson) или Спирмена (Spearman) - определяется параметром rnethod, который может иметь значение pearson или spearrnan;

3) Statistics.corr(rdd1, rdd2, method) - вычисляет корреляцию между двумя наборами RDD вещественных значений, используя алгоритм Пирсона или Спирмена - определяется параметром method, который может иметь значение pearson или spearrnan;

4) Statistics.chiSqTest(rdd) - вычисляет тест независимости Пирсона для каждого признака с маркером в наборе RDD объектов LabeledPoint. Возвращает массив объектов ChiSqTestResult, хранящих р-значение, статистики теста и степень свободы для каждого признака. Значения маркера и признака должны быть качественными величинами (то есть дискретными).

# Алгоритмы

# Линейная регрессия

In [33]:
from pyspark.mllib.regression import LinearRegressionWithSGD, LassoWithSGD, RidgeRegressionWithSGD

Данные классы имеют несколько параметров настройки алгоритма:

1) numIterations - число итераций (по умолчанию: 100);

2) stepSize - величина шага градиентного спуска (по умолчанию: 1.0);

3) intercept - определяет необходимость добавления систематического признака в данные - то есть еще одного признака, значение которого всегда равно 1 (по умолчанию false);

4) regParam - параметр регуляризации для регрессии Лассо и гребневой регрессии (по умолчанию 1.0).

In [ ]:
points = # (создать набор объектов LabeledPoint)
#Экземпляр LabeledPoint состоит из маркера label (всегда являющегося вещественным числом типа Double, но в задачах 
#классификации могущего получать дискретные целые значения) и вектора признаков features
model = LinearRegressionWithSGD.train(points, iterations=200, intercept=True)
print "weights: %s, intercept: %s" % (model.weights, model.intercept)

# Логистическая регрессия

In [34]:
from pyspark.mllib.classification import  LogisticRegressionWithLBFGS

Parameters:	

data – The training data, an RDD of LabeledPoint.

iterations – The number of iterations. (default: 100)

initialWeights – The initial weights. (default: None)

regParam – The regularizer parameter. (default: 0.0)

regType – The type of regularizer used for training our model. Supported values:
“l1” for using L1 regularization
“l2” for using L2 regularization (default)
None for no regularization

intercept – Boolean parameter which indicates the use or not of the augmented representation for training data (i.e., whether bias features are activated or not). (default: False)

corrections – The number of corrections used in the LBFGS update. If a known updater is used for binary classification, it calls the ml implementation and this parameter will have no effect. (default: 10)

tolerance – The convergence tolerance of iterations for L-BFGS. (default: 1e-6)

validateData – Boolean parameter which indicates if the algorithm should validate data before training. (default: True)

numClasses – The number of classes (i.e., outcomes) a label can take in Multinomial Logistic Regression. (default: 2)

In [35]:
data = [LabeledPoint(0.0, [0.0, 1.0]), LabeledPoint(1.0, [1.0, 0.0])]
lrm = LogisticRegressionWithLBFGS.train(sc.parallelize(data), iterations=10)
print(lrm.predict([1.0, 0.0]))
print(lrm.predict([0.0, 1.0]))

1
0


# Метод опорных векторов

In [38]:
from pyspark.mllib.classification import SVMWithSGD
data = [LabeledPoint(0.0, [0.0]), LabeledPoint(1.0, [1.0]), LabeledPoint(1.0, [2.0]), LabeledPoint(1.0, [3.0])]

svm = SVMWithSGD.train(sc.parallelize(data), iterations=10)
print svm.predict([1.0])
print svm.predict(sc.parallelize([[1.0]])).collect()
svm.clearThreshold() #сбросить порог
print svm.predict(array([1.0]))

1
[1]
1.4407421280634385


# Наивный байесовский классификатор
Параметры: lambda  - сглаживание.

При вызове модели передается набор RDD объектов LabeledPoints, где маркеры имеют значения от 0 до C - 1 при классификации
в C классов.

Возвращаемый объект NaiveBayesModel имеет метод predict (), возвращающий класс, которому лучше всего соответствует анализируемая точка. 

Также доступны два параметра обученной модели: theta, матрица вероятностей для каждого признака (размера С х D для
С классов и D признаков), и pi, С-мерный вектор приоритетов классов.

In [40]:
from pyspark.mllib.classification import NaiveBayes

# Выполнить логистическую регрессию методом SGD.
model = NaiveBayes.train(trainingData)

#Проверить положительный экземпляр (спам) и отрицательный (неспам).
#Сначала применить то же преобразование HashingTF, чтобы получить векторы признаков, затем применить модель.

posTest = tf.transform( "О М G GET cheap stuff Ьу sending money to ... ".split (" "))
negTest = tf.transform( "Hi Dad, I started studying Spark the other ... ".split(" "))
print "Prediction for positive test example: %g" %model.predict(posTest)
print "Prediction for negative test example: %g" %model.predict(negTest)

Prediction for positive test example: 1
Prediction for negative test example: 0


# Случайный лес

RandomForest принимает следующие параметры:

1) numTrees - число создаваемых деревьев. Увеличение значения numTrees уменьшает вероятность переобучения ( overfitting) на
обучающей выборке;

2) featureSubsetStrategy - число признаков, учитываемых при разбиении каждого узла; может принимать значения: auto (библиотека
сама выберет подходящее значение), all, sqrt, log2 и onethird; чем больше значение, тем выше стоимость алгоритма;

3) seed - начальное значение для генератора случайных чисел.

4) data - набор RDD объектов LabeledPoint;

5) numClasses (только для классификации) - число классов;

6) categoricalFeaturesinfo - ассоциативный массив, определяющий, какие признаки являются качественными и сколько категорий
каждый из них может иметь. Например, если признак 1 имеет двоичный характер с маркерами О и 1, а признак 2 определяет
три категории со значениями О, 1 и 2, в этом параметре нужно будет передать { 1: 2, 2: 3). Если качественных признаков
нет, передавайте пустой ассоциативный массив.

In [61]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.tree import RandomForest

dt = [LabeledPoint(0.0, [0.0]), LabeledPoint(0.0, [1.0]),LabeledPoint(1.0, [2.0]),LabeledPoint(1.0, [3.0])]

model = RandomForest.trainClassifier(sc.parallelize(dt), 2, {}, numTrees = 3, seed=42)
#model = RandomForest.trainClassifier(sc.parallelize(data), 2, {}, 3, seed=42)
print model.numTrees()
print model.totalNumNodes()
print(model)
print(model.toDebugString())

3
9
TreeEnsembleModel classifier with 3 trees

TreeEnsembleModel classifier with 3 trees

  Tree 0:
    If (feature 0 <= 1.5)
     Predict: 0.0
    Else (feature 0 > 1.5)
     Predict: 1.0
  Tree 1:
    If (feature 0 <= 1.5)
     Predict: 0.0
    Else (feature 0 > 1.5)
     Predict: 1.0
  Tree 2:
    If (feature 0 <= 0.5)
     Predict: 0.0
    Else (feature 0 > 0.5)
     Predict: 1.0



# Кластеризация

# Метод К-средних

Помимо значения К, реализация метода К-средних в MLlib имеет следующие параметры:

1) initializationMode - метод инициализации центров кластеров, который может быть "k-means||" или <<random>>; "k-means||" (по умолчанию) обычно дает лучшие результаты, но является более дорогостоящим;

2)maxIterations - максимальное число итераций (по умолчанию: 20);

3) runs - число параллельных процессов, выполняющих алгоритм.

4) rdd – Training points as an RDD of Vector or convertible sequence types.

5) k – The desired number of leaf clusters. The actual number could be smaller if there are no divisible leaf clusters. (default: 4)

6) minDivisibleClusterSize – Minimum number of points (if >= 1.0) or the minimum proportion of points (if < 1.0) of a divisible cluster. (default: 1)

7) seed – Random seed value for cluster initialization. (default: -1888008604 from classOf[BisectingKMeans].getName.##)

Реализация К-средних в MLlib поддерживает параллельное выполнение вычислений с разных начальных позиций, с выбором
лучшего результата, что позволяет получить лучшую полную модель (так как вычисления методом К-средних могут
быть остановлены в локальном минимуме).

In [5]:
from pyspark.mllib.clustering import KMeans
from numpy import array 
data = array([-383.1,-382.9, 28.7,31.2, 366.2,367.3]).reshape(6,1)
model = KMeans.train(sc.parallelize(data), 3, maxIterations=5)
model.clusterCenters

[array([366.75]), array([-383.]), array([29.95])]

# Метод чередующихся наименьших квадратов

MLlib включает реализацию метода чередующихся наименьших квадратов (Alternating Least Squares, ALS), популярного алгоритма
коллаборативной фильтрации, который прекрасно масштабируется для выполнения на кластерах. Он находится в классе mllib.recommendation.ALS.

Алгоритм ALS определяет вектор признаков для каждого пользователя и товара - такой, чтобы скалярное произведение вектора пользователя и товара было максимально близким к их оценке. 

Имеет следующие параметры:

1) rank - размер векторов признаков; чем больше значение rank, тем точнее модель, но выше стоимость вычислений (по умолчанию:
10);

2) iterations - число итераций (по умолчанию: 10);

3) lambda - регулирующий параметр (по умолчанию: 0.01);

4) alpha - константа, используемая для вычисления уровня надежности неявных оценок (по умолчанию: 1.0);

5) numUserBlocks, numProductВlocks - число блоков для разделения пользователей и товаров при параллельных вычислениях; можно
передать значение -1 (по умолчанию), чтобы библиотека MLlib могла автоматически подобрать значение для этого параметра.

In [ ]:
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating

# Load and parse the data
path = "C:\spark\SPARK_HOME\data\mllib\als\test.data"
data = sc.textFile(path)
ratings = data.map(lambda l: l.split(',')).map(lambda l: Rating(int(l[0]), int(l[1]), float(l[2])))

# Build the recommendation model using Alternating Least Squares
rank = 10
numIterations = 10
model = ALS.train(ratings, rank, numIterations)

# Evaluate the model on training data
testdata = ratings.map(lambda p: (p[0], p[1]))
predictions = model.predictAll(testdata).map(lambda r: ((r[0], r[1]), r[2]))
ratesAndPreds = ratings.map(lambda r: ((r[0], r[1]), r[2])).join(predictions)
MSE = ratesAndPreds.map(lambda r: (r[1][0] - r[1][1])**2).mean()
print("Mean Squared Error = " + str(MSE))

# Save and load model
#model.save(sc, "target/tmp/myCollaborativeFilter")
#sameModel = MatrixFactorizationModel.load(sc, "target/tmp/myCollaborativeFilter")

# Метод главных компонент (PCA)

In [7]:
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.linalg.distributed import RowMatrix

rows = sc.parallelize([
    Vectors.sparse(5, {1: 1.0, 3: 7.0}),
    Vectors.dense(2.0, 0.0, 3.0, 4.0, 5.0),
    Vectors.dense(4.0, 0.0, 0.0, 6.0, 7.0)
])

mat = RowMatrix(rows)
# Compute the top 4 principal components.
# Principal components are stored in a local dense matrix.
pc = mat.computePrincipalComponents(4) 

# Project the rows to the linear space spanned by the top 4 principal components.
projected = mat.multiply(pc) 

In [11]:
mat.rows.collect() #данные в начальной системе координат

[SparseVector(5, {1: 1.0, 3: 7.0}),
 DenseVector([2.0, 0.0, 3.0, 4.0, 5.0]),
 DenseVector([4.0, 0.0, 0.0, 6.0, 7.0])]

In [13]:
pc.values #Вектора на которые проецируются данные

array([-4.48591721e-01,  1.33019857e-01, -1.25231564e-01,  2.16507567e-01,
       -8.47651293e-01, -2.84238082e-01, -5.62115590e-02,  7.63626477e-01,
       -5.65295877e-01, -1.15603405e-01,  8.34454526e-02,  4.42397926e-02,
       -5.78071229e-01, -7.95540506e-01, -1.55011789e-01,  8.36410201e-01,
        1.72243378e-01,  2.55415489e-01,  4.85812143e-05, -4.53335549e-01])

In [15]:
projected.rows.collect() #спроецированные данные

[DenseVector([1.6486, -4.0133, -5.5245, 0.1726]),
 DenseVector([-4.6451, -1.1168, -5.5245, 0.1726]),
 DenseVector([-6.4289, -5.338, -5.5245, 0.1726])]

# Сингулярное разложение матрицы

U - ортонормальная матрица, столбцы которой называют левыми сингулярными векторами;

s - диагональная матрица с неотрицательными диагональными элементами, расположенными в порядке убывания, которые называют сингулярными значениями;

V - ортонормальная матрица, столбцы которой называют правыми сингулярными векторами.

In [16]:
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.linalg.distributed import RowMatrix

rows = sc.parallelize([
    Vectors.sparse(5, {1: 1.0, 3: 7.0}),
    Vectors.dense(2.0, 0.0, 3.0, 4.0, 5.0),
    Vectors.dense(4.0, 0.0, 0.0, 6.0, 7.0)
])

mat = RowMatrix(rows)

# Compute the top 5 singular values and corresponding singular vectors.
svd = mat.computeSVD(5, computeU=True)
U = svd.U       # The U factor is a RowMatrix.
s = svd.s       # The singular values are stored in a local dense vector.
V = svd.V       # The V factor is a local dense matrix.